In [1]:
import pandas as pd
import json
from tqdm import tqdm
import spacy
from spacy.tokens import DocBin
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
import pickle

2022-07-18 19:19:38.588693: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-07-18 19:19:38.592129: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-18 19:19:38.592141: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-07-18 19:19:48.019466: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-07-18 19:19:48.019486: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (joao): /pro

In [2]:
nlp = spacy.load('en_core_web_lg')
all_stopwords = nlp.Defaults.stop_words
all_stopwords.remove('not')

def remove_stopwords(sentence):
    text_tokens = word_tokenize(sentence)
    tokens_without_sw = [word for word in text_tokens if not word in all_stopwords]
    
    return " ".join(tokens_without_sw)

/home/joaopedrosa/miniconda3/lib/python3.9/site-packages/spacy/util.py:865: UserWarning: [W095] Model 'en_core_web_lg' (3.2.0) was trained with spaCy v3.2 and may not be 100% compatible with the current version (3.4.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [3]:
# nltk.download('punkt')
porter = PorterStemmer()

def stemSentence(sentence):
    token_words=word_tokenize(sentence)
    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(porter.stem(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)

In [4]:
# Removes StopWords and makes Stemmization
def pre_process(string):
    string = stemSentence(string)
    string = remove_stopwords(string)
    
    return string

In [9]:
data["text"]

"latin omega , omega , addit letter latin alphabet , base lowercas greek letter omega ⟨ω⟩ . wa includ latin letter mann dalbi 1982 revis african refer alphabet ha use public kulango languag côte d'ivoir 1990 . kulango public letter v hook ⟨ʋ⟩ latin upsilon ⟨ʊ⟩ found instead ."

In [8]:
with open('../data/corpus.jsonl') as f:
    with open('../data/corpus/no_stop_words/corpus_keywords_processed.jsonl', 'w') as f2:
        for line in tqdm(f):
            data     = json.loads(line)
            
            # Preprocessing
            data["title"]    = pre_process(data["title"])
            data["text"]     = pre_process(data["text"])
            data["keywords"] = [pre_process(k) for k in data["keywords"]]
            
            # Storing
            new_data = {"id": data["id"], "contents": data["title"] + "\n" + data["text"] + "\n"}
            for k in data["keywords"]:
                new_data["contents"] += k + "\n"
            n_string = json.dumps(new_data)
            f2.write(n_string)
            f2.write('\n')

4641784it [2:09:31, 597.28it/s] 


In [67]:
with open('../data/corpus/no_stop_words/corpus_keywords_processed.jsonl') as f:
    with open('../data/corpus_pyt.jsonl', 'w') as f2:
        for line in tqdm(f):
            data     = json.loads(line)
            new_data = {"docno": data["id"], "text": data["contents"]}
            
            n_string = json.dumps(new_data)
            f2.write(n_string)
            f2.write('\n')

4641784it [00:29, 158276.96it/s]


In [ ]:
with open('../data/corpus.jsonl') as f:
    with open('../data/corpus/no_stop_words/corpus_keywords_processed.jsonl', 'w') as f2:
        for line in tqdm(f):
            data     = json.loads(line)
            # Storing
            new_data = {"docno": data["id"], "contents": data[""] + "\n" + data["text"] + "\n"}
            for k in data["keywords"]:
                new_data["contents"] += k + "\n"
            n_string = json.dumps(new_data)
            f2.write(n_string)
            f2.write('\n')

In [41]:
# We're gonna use or Spacy Neural Network to assist us in the ranking

def answer(searcher, q):
    data = {"QueryId":[], "EntityId":[]}
    # data = {"QueryId":[], "EntityId":[], "Score":[]}hmmm então
    for i in range(len(q)):
        qidx = q["QueryId"][i]
        hits = searcher.search(pre_process(q["Query"][i]), k = 100)
        for hit in hits:
            data["QueryId"].append(str(qidx).zfill(3))
            data["EntityId"].append(hit.docid)
            # data["Score"].append(hit.score)
    
    return pd.DataFrame.from_dict(data)

In [12]:
tq = pd.read_csv("../data/test_queries.csv")

In [6]:
searcher = LuceneSearcher('../data/indexes/sparse_no_stop/')

In [15]:
h = searcher.search(pre_process("2001 debut album"))

In [31]:
for hit in h:
    print(hit.docid)
    print(hit.score)
    print(hit.raw)

1912152
8.619500160217285
{
  "id" : "1912152",
  "contents" : "seriou\nseriou debut studio album american rapper t.i . , releas octob 9 , 2001 arista record . album spawn eponym singl , featur jamaican regga entertain beeni man . hi debut singl , \\i seriou , \\ wa releas june 26 , 2001 . singl receiv littl airplay fail chart .\n2001 debut album\nalbum produc dj toomp\nalbum produc jazz pha\nalbum produc lil jon\nalbum produc neptun\narista record album\n"
}
0000002
8.617500305175781
{
  "id" : "0000002",
  "contents" : "! ! ! ( album )\n! ! ! eponym debut album ! ! ! . wa releas 2001 gold standard laboratori .\n! ! ! album\n2001 debut album\nenglish-languag album\n"
}
1894433
8.552900314331055
{
  "id" : "1894433",
  "contents" : "humanist ( album )\nhumanist debut album abandon pool . wa releas septemb 2001 .\n2001 debut album\nextasi record album\ntommi walter album\n"
}
4151319
8.514100074768066
{
  "id" : "4151319",
  "contents" : "voic ( russel watson album )\nvoic 2001 debut al

In [34]:
tq.to_csv("../data/sample_queries.tsv", sep = "\t")

In [42]:
df_ans = answer(searcher, tq)

In [43]:
df_ans

,QueryId,EntityId
0,002,0366601
1,002,3572540
2,002,0758546
3,002,2508826
4,002,1850915
...,...,...
23200,465,3543283
23201,465,1470065
23202,465,4537965
23203,465,0103926


In [44]:
df_ans.to_csv("../data/results/20-57.csv", index=False)

In [27]:
hits = searcher.search('potato potato', k = 100)

for i in range(len(hits)):
    print(f'{i+1:2} {hits[i].docid:4} {hits[i].score:.5f}')

 1 1537119 13.91340
 2 3481031 13.83070
 3 3357186 13.62200
 4 3357207 13.58150
 5 2989663 13.52220
 6 4463109 13.48630
 7 3357185 13.48290
 8 1464351 13.40500
 9 3357195 13.40500
10 3357214 13.40500
11 0650179 13.36640
12 2552426 13.36640
13 3357184 13.36170
14 3357216 13.32810
15 0989109 13.28990
16 2542131 13.28990
17 3357206 13.28990
18 3373092 13.28990
19 3357182 13.25200
20 0614032 13.24730
21 3357211 13.24730
22 0332427 13.20210
23 0527265 13.20210
24 2398754 13.20210
25 2454875 13.20210
26 4551472 13.20210
27 3131868 13.18400
28 3357197 13.17670
29 1548007 13.15720
30 3357223 13.15720
31 0719244 13.07260
32 4538071 13.06830
33 2735064 13.04510
34 3612207 13.04510
35 0343050 13.02430
36 0964477 13.02430
37 3357187 13.02430
38 0534861 12.98060
39 1741349 12.98060
40 3357149 12.98060
41 3357164 12.98060
42 3357178 12.98060
43 3357179 12.97020
44 0805047 12.96310
45 3357181 12.96310
46 3357158 12.93720
47 3357188 12.93400
48 1973148 12.90900
49 2735061 12.89410
50 3357198 12.89410


### Let's make a Neural Network 8)
First we're gonna build the dataset then do the training with python -m shennaningans

In [31]:
# Let's throw stuff into memory. This uses huge memory so I'll run it once then throw only useful information into a CSV
# in a way that I can retrieve information later without sending it all to memory.

# corpus = {}
# with open('../data/corpus.jsonl') as f:
#     for line in tqdm(f):
#         data               = json.loads(line)
#         corpus[data['id']] = data
        
# with open('../data/corpus.pickle', 'wb') as handle:
#     pickle.dump(corpus, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Loading
with open('../data/corpus.pickle', 'rb') as handle:
    corpus = pickle.load(handle)

# Reading training queries
train_queries = pd.read_csv("../data/train_queries.csv")
qdict = {train_queries["QueryId"][i]:train_queries["Query"][i] for i in range(len(train_queries))}

train_answers = pd.read_csv("../data/train_qrels.csv")

# Building a complete csv
train_dict = {"Text":[], "Class":[]}

for i in tqdm(range(len(train_answers))):
    data = corpus[str(train_answers['EntityId'][i]).zfill(7)]
    train_dict["Text"].append(pre_process(qdict[train_answers["QueryId"][i]]) + "\n" + \
                              pre_process(data["title"]) + "\n" + \
                              pre_process(data["text"]) + "\n" + \
                              "\n".join([pre_process(k) for k in data["keywords"]]))
    
    train_dict["Class"].append(train_answers["Relevance"][i])
    
train_df.to_csv("../data/train_data.csv", index = False)
train_df = pd.DataFrame.from_dict(train_dict)

100%|██████████████████████████████████████| 8202/8202 [00:17<00:00, 460.01it/s]


In [2]:
train_df = pd.read_csv("../data/train_data.csv")

In [14]:
# Let's transform data into Spacy docs
# nlp is spacy.load("en_core_web_lg")

'''
Receives Tuples and return Spacy Docs.

Input:
    data: List of tuples of format (Text, Class)
    classes: List or Set of possible classes - Set is Faster
    nlp: Spacy model to use. If you don't know what to use, for portuguese documentos you should use "nlp = spacy.load("pt_core_news_lg")

Output:
    A list of spacy Docs
'''
def make_spacy_docs(data, classes, nlp):
    docs = []

    # Here the text is already added to a doc with the nlp.pip function
    for doc, label in tqdm(nlp.pipe(data, as_tuples=True)):
        if label not in classes:
            raise Exception("One of your examples in data have a label that is not listed in the possible classes.")
        # Create label for each class
        for c in classes:
            if(c != label):
                doc.cats[c] = 0
            else:
                doc.cats[c] = 1
        # Add document to return list
        docs.append(doc)

    return docs

In [5]:
train_df

,Text,Class
0,europ solar power facil\nandasol solar power s...,1
1,europ solar power facil\narchimed solar power ...,2
2,europ solar power facil\nbp solar\nbp solar wa...,1
3,europ solar power facil\nbavaria solarpark\nba...,2
4,europ solar power facil\nbeeckerwerth\nbeecker...,2
...,...,...
8197,member beaux art trio\ndaniel guilet\ndaniel g...,1
8198,member beaux art trio\nida kavafian\nida kavaf...,2
8199,member beaux art trio\nisidor cohen\ncompos bo...,1
8200,member beaux art trio\nmenahem pressler\nmenah...,2


In [6]:
classes = {"partially_relevant", "highly_relevant"}
train_tuples = []
for i in range(len(train_df)):
    text = train_df["Text"][i]
    c    = "partially_relevant" if train_df["Class"][i] == 1 else "highly_relevant"
    train_tuples.append((text, c))

In [17]:
# We're not gonna shuffle to keep independent queries in Validation
TRAIN = int(0.8 * len(train_tuples))

# TRAIN
train_docs = make_spacy_docs(train_tuples[:TRAIN], classes, nlp)
doc_bin    = DocBin(docs = train_docs)
doc_bin.to_disk("../data/spacy/train.spacy")

# VAL
val_docs = make_spacy_docs(train_tuples[TRAIN:], classes, nlp)
doc_bin  = DocBin(docs = val_docs)
doc_bin.to_disk("../data/spacy/val.spacy")

6561it [00:37, 174.88it/s]
1641it [00:09, 165.56it/s]


Now we use python -m spacy train to train our model and then we go to next steps:

In [61]:
nlp = spacy.load("../data/spacy/models/07-18-tf/model-best/")

/home/joaopedrosa/miniconda3/lib/python3.9/site-packages/spacy/util.py:865: UserWarning: [W095] Model 'en_pipeline' (0.0.0) was trained with spaCy v3.3 and may not be 100% compatible with the current version (3.4.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [62]:
# We're gonna use or Spacy Neural Network to assist us in the ranking

def answer(searcher, q, nlp):
    data = {"QueryId":[], "EntityId":[]}
    # data = {"QueryId":[], "EntityId":[], "Score":[]}
    for i in tqdm(range(len(q))):
        qidx  = q["QueryId"][i]
        qtext = pre_process(q["Query"][i])
        hits = searcher.search(qtext, k = 100)
        
        prel = []
        hrel = []
        for hit in hits:
            # Passing through network
            text = qtext+"\n"+ json.loads(hit.raw)["contents"]+"\n"
            doc = nlp(text)
            
            if(doc.cats['highly_relevant'] > 0.5):
                hrel.append([str(qidx).zfill(3), hit.docid])
            else:
                prel.append([str(qidx).zfill(3), hit.docid])
        
        # Highly Relevant First
        for qid, eid in hrel:
            data["QueryId"].append(qid)
            data["EntityId"].append(eid)
        
        # Partially Relevant After
        for qid, eid in prel:
            data["QueryId"].append(qid)
            data["EntityId"].append(eid)
            
    return pd.DataFrame.from_dict(data)

In [63]:
df_ans = answer(searcher, tq, nlp)

  0%|                                                   | 0/233 [00:00<?, ?it/s]/home/joaopedrosa/miniconda3/lib/python3.9/site-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
100%|█████████████████████████████████████████| 233/233 [42:08<00:00, 10.85s/it]


In [65]:
df_ans.to_csv("../data/results/spacy-transformer_18-35.csv", index=False)

In [58]:
# This is the same but we order based only in the relevance probability of the network

def answer(searcher, q, nlp):
    data = {"QueryId":[], "EntityId":[]}
    # data = {"QueryId":[], "EntityId":[], "Score":[]}
    for i in tqdm(range(len(q))):
        qidx  = q["QueryId"][i]
        qtext = pre_process(q["Query"][i])
        hits = searcher.search(qtext, k = 100)
        
        h = []
        for hit in hits:
            # Passing through network
            text = qtext+"\n"+ json.loads(hit.raw)["contents"]+"\n"
            doc = nlp(text)
            # We use partially_relevant here because it is basically (1 - highly_relevant) and we will sort it by increasing order.
            h.append([doc.cats['partially_relevant'], str(qidx).zfill(3), hit.docid])
            
        h = sorted(h)
        
        # Ordered by confidence on highly_relevant class
        for _, qid, eid in h:
            data["QueryId"].append(qid)
            data["EntityId"].append(eid)
        
    return pd.DataFrame.from_dict(data)

In [59]:
df_ans = answer(searcher, tq, nlp)

100%|█████████████████████████████████████████| 233/233 [08:55<00:00,  2.30s/it]


In [60]:
df_ans.to_csv("../data/results/spacy-ordered_16-01.csv", index=False)

## Okay let's try Pyterrier

In [6]:
def iter_file(filename):
    import json
    with open(filename, 'rt') as file:
        for l in file:
            # assumes that each line contains 'docno', 'text' attributes
            # yields a dictionary for each json line
            yield json.loads(l)

# indexref4 = pt.IterDictIndexer("./index", meta=['docno', 'text'], meta_lengths=[20, 4096]).index(iter_file("/path/to/file.jsonl"))

In [6]:
import pyterrier as pt
pt.init()

PyTerrier 0.8.1 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [7]:
index = pt.IterDictIndexer("../data/indexes/pyterrier", meta={'docno': 20, 'text': 1024}, type=pt.index.IndexingType(1), overwrite=True, verbose=True)
index = index.index(iter_file("../data/corpus_pyt.jsonl"), fields = ['text'])

18:52:55.010 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (4641604) - further warnings are suppressed
18:52:55.037 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer - Indexed 1 empty documents


In [13]:
tq = pd.read_csv("../data/test_queries.csv")
tq.columns = ["qid", "query"]

for i in range(len(tq)):
    text = tq["query"][i]
    text = pre_process(text)
    text = text.replace("'","")
    text = text.replace("!", "")
    tq["query"][i] = text

/tmp/ipykernel_78440/106473622.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tq["query"][i] = text


In [42]:
# dataset = bm25.transform(tq)

In [31]:


# dataset = pt.get_dataset("irds:vaswani")
# bm25 = pt.BatchRetrieve(pt.get_dataset("vaswani").get_index(), wmodel="BM25")
# mono_pipeline = bm25 >> pt.text.get_text(dataset, "text") >> monoT5
# duo_pipeline = mono_pipeline % 50 >> duoT5 # apply a rank cutoff of 50 from monoT5 since duoT5 is too costly to run over the full result list

/home/joaopedrosa/miniconda3/lib/python3.9/site-packages/transformers/models/t5/tokenization_t5.py:164: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [32]:
ds = bm25.transform(tq)

In [ ]:
# This is the same but we order based only in the relevance probability of the network

def answer(searcher, q, nlp):
    data = {"QueryId":[], "EntityId":[]}
    # data = {"QueryId":[], "EntityId":[], "Score":[]}
    for i in tqdm(range(len(q))):
        qidx  = q["QueryId"][i]
        qtext = pre_process(q["Query"][i])
        hits = searcher.search(qtext, k = 100)
        
        h = []
        for hit in hits:
            # Passing through network
            text = qtext+"\n"+ json.loads(hit.raw)["contents"]+"\n"
            doc = nlp(text)
            # We use partially_relevant here because it is basically (1 - highly_relevant) and we will sort it by increasing order.
            h.append([doc.cats['partially_relevant'], str(qidx).zfill(3), hit.docid])
            
        h = sorted(h)
        
        # Ordered by confidence on highly_relevant class
        for _, qid, eid in h:
            data["QueryId"].append(qid)
            data["EntityId"].append(eid)
        
    return pd.DataFrame.from_dict(data)

In [33]:
ds

,qid,docid,docno,text,rank,score,query
0,2,366600,0366601,ancient roman architecturancient roman archite...,0,22.224013,roman architectur
1,2,2508825,2508826,list ancient architectur recordlist ancient ar...,1,21.091674,roman architectur
2,2,3572539,3572540,roman architectur revolutroman architectur rev...,2,21.075968,roman architectur
3,2,1850914,1850915,histori roman byzantin domehistori roman byzan...,3,20.623313,roman architectur
4,2,3957053,3957054,"sudatoriumarchitectur , sudatorium vault sweat...",4,20.213101,roman architectur
...,...,...,...,...,...,...,...
23200,465,3361887,3361888,praemium imperialpraemium imperial ( lit . \wo...,95,32.834404,organ award nobel prize
23201,465,2350828,2350829,kyoto prize advanc technologkyoto prize advanc...,96,32.804454,organ award nobel prize
23202,465,291347,0291348,albert lasker award basic medic researchalbert...,97,32.784861,organ award nobel prize
23203,465,2946653,2946654,mdecin san frontirmdecin san frontir ( msf ) (...,98,32.777955,organ award nobel prize


In [34]:
from pyterrier_t5 import MonoT5ReRanker
monoT5 = MonoT5ReRanker(batch_size = 8) # loads castorini/monot5-base-msmarco by default

index = pt.IndexFactory.of("../data/indexes/pyterrier/")

bm25 = pt.BatchRetrieve(index, wmodel="BM25", metadata=["docno", "text"], num_results = 100)
mono_pipeline = bm25 >> monoT5

19:43:35.863 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 943,1 MiB of memory would be required.


In [38]:
data = {"QueryId":[], "EntityId":[]}
qc = {}
for i in range(len(ds)):
    qid   = str(ds["qid"][i]).zfill(3)
    eid   = ds["docno"][i]
    
    data["QueryId"].append(qid)
    data["EntityId"].append(eid)

df_answer = pd.DataFrame.from_dict(data)
df_answer.to_csv("../data/results/pyterrier_t5.csv", index = False)

In [42]:
df_answer.to_csv("../data/results/19-54.csv", index = False)

In [43]:
ds = mono_pipeline.transform(tq)

monoT5: 100%|████████████████████████| 2901/2901 [1:05:01<00:00,  1.34s/batches]


In [52]:
data = {"QueryId":[], "EntityId":[], "Rank":[]}
qc = {}
for i in range(len(ds)):
    qid   = str(ds["qid"][i]).zfill(3)
    eid   = ds["docno"][i]
    
    data["QueryId"].append(qid)
    data["EntityId"].append(eid)
    data["Rank"].append(ds["rank"][i])

df_answer = pd.DataFrame.from_dict(data)
# df_answer.to_csv("../data/results/pyterrier_t5.csv", index = False)

In [45]:
ds2 = ds

In [56]:
df_final = df_answer.sort_values(by=['QueryId', 'Rank'])

In [59]:
df_final.drop(['Rank'], axis=1).to_csv("../data/results/21-09.csv", index = False)

In [47]:
ds.sort_values(by=['qid', 'rank'])

,qid,docid,docno,text,query,score,rank
5006,101,2073083,2073084,jean de wavrinjehan ( jean ) de waurin ( wavri...,nobl english person hundr year war,-0.018614,0
5047,101,1896018,1896019,hundr year ' warhundr year ' war wa seri confl...,nobl english person hundr year war,-0.028138,1
5002,101,1896021,1896022,hundr year ' war ( 141553 )lancastrian war wa ...,nobl english person hundr year war,-0.031801,2
5025,101,4182853,4182854,hundr thirti year ' warhundr thirti year ' war...,nobl english person hundr year war,-0.038529,3
5068,101,1896019,1896020,"hundr year ' war ( 133760 )hundr year ' war , ...",nobl english person hundr year war,-0.041457,4
...,...,...,...,...,...,...,...
4957,97,2609831,2609832,ludila damludila dam graviti dam jinsha river ...,famou river confluenc dam construct,-11.032598,95
4962,97,4337952,4337953,upper atbara setit dam complexupper atbara set...,famou river confluenc dam construct,-11.216054,96
4989,97,1956259,1956260,indu basin projectindu basin project water con...,famou river confluenc dam construct,-12.051404,97
4982,97,4562246,4562247,xiaonanhai damxiaonanhai dam wa propos dam yan...,famou river confluenc dam construct,-12.073783,98
